In [3]:
import random

import numpy as np
import torch
from torch.backends import cudnn

# Random seed to maintain reproducible results
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
# Use GPU for training by default
device = torch.device("cuda", 0)
# Turning on when the image size does not change during training can speed up training
cudnn.benchmark = True
# When evaluating the performance of the SR model, whether to verify only the Y channel image data
only_test_y_channel = True
# Model architecture name
d_arch_name = "discriminator"
g_arch_name = "srresnet_x4"
# Model arch config
in_channels = 3
out_channels = 3
channels = 64
num_rcb = 16
# Test upscale factor
upscale_factor = 4
# Current configuration parameter method
mode = "test"
# Experiment name, easy to save weights and log files
exp_name = "SRGAN_x4-Set5"

if mode == "train":
    # Dataset address
    train_gt_images_dir = f"./data/ImageNet/SRGAN/train"

    test_gt_images_dir = f"./data/Set5/GTmod12"
    test_lr_images_dir = f"./data/Set5/LRbicx{upscale_factor}"

    gt_image_size = 96
    batch_size = 16
    num_workers = 4

    # The address to load the pretrained model
    pretrained_d_model_weights_path = f""
    pretrained_g_model_weights_path = f"./results/SRResNet_x4-DIV2K/g_last.pth.tar"

    # Incremental training and migration training
    resume_d_model_weights_path = f""
    resume_g_model_weights_path = f""

    # Total num epochs (200,000 iters)
    epochs = 18

    # Loss function weight
    pixel_weight = 1.0
    content_weight = 1.0
    adversarial_weight = 0.001

    # Feature extraction layer parameter configuration
    feature_model_extractor_node = "features.35"
    feature_model_normalize_mean = [0.485, 0.456, 0.406]
    feature_model_normalize_std = [0.229, 0.224, 0.225]

    # Optimizer parameter
    model_lr = 1e-4
    model_betas = (0.9, 0.999)
    model_eps = 1e-8
    model_weight_decay = 0.0

    # Dynamically adjust the learning rate policy [100,000 | 200,000]
    lr_scheduler_step_size = epochs // 2
    lr_scheduler_gamma = 0.1

    # How many iterations to print the training result
    train_print_frequency = 100
    valid_print_frequency = 1

if mode == "test":
    # Test data address
    lr_dir = f"./data/Set5/LRbicx{upscale_factor}"
    sr_dir = f"./results/test/{exp_name}"
    gt_dir = f"./data/Set5/GTmod12"

    g_model_weights_path = f"./results/pretrained_models/SRGAN_x4-ImageNet-8c4a7569.pth.tar"

    


In [23]:
import os
import cv2
import torch
import model
from natsort import natsorted
from numpy import ndarray

def image_to_tensor(image: ndarray, range_norm: bool, half: bool) -> Tensor:
    # Convert image data type to Tensor data type
    tensor = torch.from_numpy(np.ascontiguousarray(image)).permute(2, 0, 1).float()

    # Scale the image data from [0, 1] to [-1, 1]
    if range_norm:
        tensor = tensor.mul(2.0).sub(1.0)

    # Convert torch.float32 image data type to torch.half image data type
    if half:
        tensor = tensor.half()

    return tensor

def tensor_to_image(tensor: Tensor, range_norm: bool, half: bool) -> Any:
    if range_norm:
        tensor = tensor.add(1.0).div(2.0)
    if half:
        tensor = tensor.half()

    image = tensor.squeeze(0).permute(1, 2, 0).mul(255).clamp(0, 255).cpu().numpy().astype("uint8")

    return image


def preprocess_one_image(image_path: str, device: torch.device) -> Tensor:
    image = cv2.imread(image_path).astype(np.float32) / 255.0

    # BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert image data to pytorch format data
    tensor = image_to_tensor(image, False, False).unsqueeze_(0)

    # Transfer tensor channel image format data to CUDA device
    tensor = tensor.to(device=device, memory_format=torch.channels_last, non_blocking=True)

    return tensor

model_names = sorted(
    name for name in model.__dict__ if
    name.islower() and not name.startswith("__") and callable(model.__dict__[name]))


def main() -> None:
    # Initialize the super-resolution bsrgan_model
    g_model = model.__dict__[g_arch_name](in_channels=in_channels,
                                                       out_channels=out_channels,
                                                       channels=channels,
                                                       num_rcb=num_rcb)
    g_model = g_model.to(device=device)
    print(f"Build `{g_arch_name}` model successfully.")

    # Load the super-resolution bsrgan_model weights
    checkpoint = torch.load(g_model_weights_path, map_location=lambda storage, loc: storage)
    g_model.load_state_dict(checkpoint["state_dict"])
    print(f"Load `{g_arch_name}` model weights "
          f"`{os.path.abspath(g_model_weights_path)}` successfully.")

    # Create a folder of super-resolution experiment results
    #make_directory(srgan_config.sr_dir)

    # Start the verification mode of the bsrgan_model.
    g_model.eval()

    # Get a list of test image file names.
    file_names = natsorted(os.listdir(lr_dir))
    # Get the number of test image files.
    total_files = len(file_names)

    for index in range(total_files):
        lr_image_path = os.path.join(lr_dir, file_names[index])
        sr_image_path = os.path.join(sr_dir, file_names[index])
        gt_image_path = os.path.join(gt_dir, file_names[index])

        print(f"Processing `{os.path.abspath(lr_image_path)}`...")
        lr_tensor = preprocess_one_image(lr_image_path, device)
        gt_tensor = preprocess_one_image(gt_image_path, device)

        # Only reconstruct the Y channel image data.
        with torch.no_grad():
            sr_tensor = g_model(lr_tensor)

        # Save image
        sr_image = tensor_to_image(sr_tensor, False, False)
        sr_image = cv2.cvtColor(sr_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(sr_image_path, sr_image)

if __name__ == "__main__":
    main()


Build `srresnet_x4` model successfully.
Load `srresnet_x4` model weights `F:\A研究生\计算机视觉与应用实践\计算机视觉实践-练习3\SRGAN\results\pretrained_models\SRGAN_x4-ImageNet-8c4a7569.pth.tar` successfully.
Processing `F:\A研究生\计算机视觉与应用实践\计算机视觉实践-练习3\SRGAN\data\Set5\LRbicx4\baby.png`...
Processing `F:\A研究生\计算机视觉与应用实践\计算机视觉实践-练习3\SRGAN\data\Set5\LRbicx4\bird.png`...
Processing `F:\A研究生\计算机视觉与应用实践\计算机视觉实践-练习3\SRGAN\data\Set5\LRbicx4\butterfly.png`...
Processing `F:\A研究生\计算机视觉与应用实践\计算机视觉实践-练习3\SRGAN\data\Set5\LRbicx4\head.png`...
Processing `F:\A研究生\计算机视觉与应用实践\计算机视觉实践-练习3\SRGAN\data\Set5\LRbicx4\woman.png`...


In [11]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 加载原始高分辨率图像和生成图像
original_img = Image.open('data/Set5/GTmod12/baby.png').convert('RGB')
generated_img = Image.open('results/test/SRGAN_x4-Set5/baby.png').convert('RGB')

# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_baby:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_baby:', ssim_score)

PSNR_baby: 30.642098472244918
SSIM_baby: 0.7820139


In [12]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 加载原始高分辨率图像和生成图像
original_img = Image.open('data/Set5/GTmod12/bird.png').convert('RGB')
generated_img = Image.open('results/test/SRGAN_x4-Set5/bird.png').convert('RGB')

# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_bird:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_bird:', ssim_score)

PSNR_bird: 29.808428251277146
SSIM_bird: 0.80076116


In [13]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 加载原始高分辨率图像和生成图像
original_img = Image.open('data/Set5/GTmod12/butterfly.png').convert('RGB')
generated_img = Image.open('results/test/SRGAN_x4-Set5/butterfly.png').convert('RGB')

# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_butterfly:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_butterfly:', ssim_score)

PSNR_butterfly: 25.255649471235454
SSIM_butterfly: 0.7334442


In [14]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 加载原始高分辨率图像和生成图像
original_img = Image.open('data/Set5/GTmod12/head.png').convert('RGB')
generated_img = Image.open('results/test/SRGAN_x4-Set5/head.png').convert('RGB')

# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_head:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_head:', ssim_score)

PSNR_head: 28.82089678146748
SSIM_head: 0.6070606


In [15]:
import torch
import math
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
# 加载原始高分辨率图像和生成图像
original_img = Image.open('data/Set5/GTmod12/woman.png').convert('RGB')
generated_img = Image.open('results/test/SRGAN_x4-Set5/woman.png').convert('RGB')

# 将图像转换为PyTorch张量
original_img = transforms.ToTensor()(original_img).unsqueeze(0)
generated_img = transforms.ToTensor()(generated_img).unsqueeze(0)

# 计算PSNR
max_value = 1.0  # 假设像素值的范围为[0,1]
mse = torch.mean((generated_img - original_img) ** 2)
psnr = 10 * math.log10(max_value**2 / mse)

print('PSNR_woman:', psnr)

# 计算 SSIM
original_img = original_img.squeeze().permute(1,2,0).detach().numpy()
generated_img = generated_img.squeeze().permute(1,2,0).detach().numpy()
ssim_score = ssim(original_img,generated_img,win_size=3,multichannel=True,channel_axis=2,data_range=generated_img.max() - generated_img.min())

print('SSIM_woman:', ssim_score)

PSNR_woman: 27.75483025739634
SSIM_woman: 0.8140816
